### Traditional Model Solution

In [1]:
import math
import numpy
from gurobipy import *
import os

'''
supply chain configuration
'''
c = 0
h_List = [182,660,1155,465,46,218,270,345,1292,2235,2770,1350,195,2129,8697,2775,172,27,900,17201,1279,21780]
b_List = [1000,1000,1000,1000,1000,1000,1000,1000,5000,1000,5000,1000,1000,5000,10000,5000,1000,1000,1000,20000,5000,40000]
L_List = [19,15,8,9,9,6,9,8,16,15,6,7,2,7,5,10,11,35,10,8,7,4]

N = 22
Structure = [[-1],[-1],[-1],[-1],[-1],[-1],[-1],[-1],[0,1],[-1],[2,3,4],[-1],[-1],[5,6,7],[8,9,10],[11,12],[-1],[-1],[-1],[13,14,15],[16,17],[18,19,20]]
S_in_supplier = [10,10,10,10,10,10,10,10,1e20,10,1e20,10,10,1e20,1e20,1e20,10,10,10,1e20,1e20,1e20]
S_out_customer = [1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,5]

Non_Start = [] #find the set of stages with upstream stages
for j in range (N):
    if (Structure[j][0] > -1):
        Non_Start.append(j)

for i in range (N): #re-scaling
    b_List[i] = b_List[i] * 100 / 100.0
    h_List[i] = h_List[i] / 100.0

'''
distribution used in model formulation
'''
probs_Z = [0.33,0.34,0.33]
values_Z = [10,20,30] #Z
mu_model_Z = 0
mu_sq_model_Z= 0
mu_model = 0 #demand mean
mu_sq_model= 0
for pre_i in range (len(probs_Z)):
    mu_model_Z += probs_Z[pre_i]*values_Z[pre_i]
    mu_sq_model_Z += probs_Z[pre_i]*values_Z[pre_i]*values_Z[pre_i]
    mu_model += probs_Z[pre_i]/values_Z[pre_i]
    mu_sq_model += probs_Z[pre_i]/values_Z[pre_i]/values_Z[pre_i]
sigma_Z = math.sqrt(mu_sq_model_Z-mu_model_Z*mu_model_Z)
sigma = math.sqrt(mu_sq_model-mu_model*mu_model) #demand std

'''
calculate optimal solution
'''
S_MAX = 101
           
def optimal_solution (k):
    F_S_S = []
    solution = []
    for S_in in range (0,S_MAX):
        temp = []
        for S_out in range (0,S_MAX):
            SubCost = []
            for j in range (N):
                if ((S_in+L_List[j]-S_out) >= 0):
                    SubCost.append(h_List[j]*k*sigma*math.sqrt(S_in+L_List[j]-S_out))
                else: 
                    SubCost.append(5000)
            temp.append(SubCost)
        F_S_S.append(temp)

    F_S_S = numpy.array(F_S_S)
    print("F_S_S generated!")

    #Solve MILP
    try:
        m = Model("Algo-Trad")
        # Create variables
        chi = m.addVars(N, S_MAX, S_MAX, vtype=GRB.BINARY, name="chi")
        # Set objective
        obj = 0
        for j in range (N):
            for mu in range (S_MAX):
                for nu in range (S_MAX):
                    obj += F_S_S[mu,nu,j]*chi[j,mu,nu]
        m.setObjective(obj, GRB.MINIMIZE)
        # Add constraint
        for j in range (N):
            m.addConstr(quicksum(chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == 1)
            if (S_out_customer[j] < 1e20):
                m.addConstr(quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_out_customer[j])
            if (S_in_supplier[j] < 1e20):
                m.addConstr(quicksum(mu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_in_supplier[j])
        for i in range (len(Non_Start)):
            for j in Structure[Non_Start[i]]:
                m.addConstr(quicksum(mu*chi[Non_Start[i],mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) >= quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)))

        m.optimize()

    except GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')

    for j in range (N):
        for mu in range (S_MAX):
            for nu in range (S_MAX):
                if(chi[j,mu,nu].X == 1):
                    solution.append([F_S_S[mu,nu,j],mu,nu]) #obj,S_in,S_out
    
    return solution.copy()

Trad_General = open("Trad-General-Simulation.txt", mode='w')
Trad_Poisson = open("Trad-Poisson-Simulation.txt", mode='w')
Trad_Solution = open("Trad-Solution.txt", mode='w')
for k in [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,
         2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,
          3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,
         4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0]:
    opt_temp = optimal_solution(k)
    opt_temp = numpy.array(opt_temp)
    S_in_List = opt_temp[:,1]
    S_out_List = opt_temp[:,2]
    S_in_List = S_in_List.tolist()
    S_out_List = S_out_List.tolist()
    W_List = S_in_List
    B_List = []
    for i in range (N):
        B_List.append(round(mu_model*(S_in_List[i]+L_List[i]-S_out_List[i])+k*sigma*math.sqrt(S_in_List[i]+L_List[i]-S_out_List[i]))) 
    Trad_Solution.write(str(k)+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_in_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_out_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(B_List[i])+'\t')
    Trad_Solution.write('\n') 
    #simulation-general
    for w in range (100):
        numpy.random.seed(w+10)
        Z = numpy.random.choice(values_Z,500,p=probs_Z).tolist() #Z
        demand = [] #real-time demand 0 or 1
        for z in Z:
            for z_i in range (z-1):
                demand.append(0)
            demand.append(1)
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_General.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_General.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_General.write(str(backordertimes[i])+'\t')
        Trad_General.write('\n')
    print(k, "Simulation-General Ended!")
    #simulation-poisson
    for w in range (100):
        numpy.random.seed(w+10)
        demand = numpy.random.poisson(mu_model,int(500*mu_model_Z)).tolist()
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_Poisson.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_Poisson.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_Poisson.write(str(backordertimes[i])+'\t')
        Trad_Poisson.write('\n')
    print(k, "Simulation-Poisson Ended!") 
Trad_General.close() #every row: k,w,cost of stages 0~21, backorder times of stages 0~21
Trad_Poisson.close()
Trad_Solution.close() #every row: k, inbound service time of stages 0~21, outbound service time of stages 0~21, base stock of stages 0~21

F_S_S generated!

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-07-13
Using license file /Users/estelle/gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x846f1066
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [8e-03, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 89.1622915
Presolve removed 15 rows and 199464 columns (presolve time = 9s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 9.43s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 72.9778068
Variable types: 0 continuous, 23908 integer (23908 b

Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [9e-03, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 106.9947498
Presolve removed 15 rows and 199464 columns (presolve time = 8s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 8.02s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 87.5733681
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      8s
      85    2.5005568e+01   0.000000e+00   0.000000e+00      8s

Root relaxation: objective 2.500557e+01, 85 iterations, 0.03 seconds
Total elapsed time = 8.45s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   25.00557    0   30   87.

       0    0.0000000e+00   2.100000e+01   0.000000e+00      8s
      85    2.9173163e+01   0.000000e+00   0.000000e+00      8s

Root relaxation: objective 2.917316e+01, 85 iterations, 0.06 seconds
Total elapsed time = 8.36s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   29.17316    0   30  102.16893   29.17316  71.4%     -    8s
H    0     0                      69.8637921   29.17316  58.2%     -    8s
H    0     0                      55.6236933   29.17316  47.6%     -    8s
H    0     0                      54.8497335   29.17316  46.8%     -    8s
     0     0   33.97103    0   34   54.84973   33.97103  38.1%     -    8s
H    0     0                      51.4422572   33.97103  34.0%     -    8s
     0     0   35.01284    0   33   51.44226   35.01284  31.9%     -    8s
     0     0   40.64956    0   57   51.44226   40.64956  21.0%     -    8s
     0     0   40.64956

     0     0   38.82404    0   34   62.68541   38.82404  38.1%     -    8s
H    0     0                      58.7911511   38.82404  34.0%     -    8s
     0     0   40.01468    0   33   58.79115   40.01468  31.9%     -    8s
     0     0   46.45664    0   57   58.79115   46.45664  21.0%     -    9s
     0     0   46.45664    0   31   58.79115   46.45664  21.0%     -    9s
     0     0   46.45664    0   49   58.79115   46.45664  21.0%     -    9s
H    0     0                      58.7758849   46.45664  21.0%     -    9s
     0     0   47.17889    0   55   58.77588   47.17889  19.7%     -    9s
     0     0   47.33222    0   53   58.77588   47.33222  19.5%     -    9s
     0     0   47.33222    0   54   58.77588   47.33222  19.5%     -    9s
     0     0   50.64170    0   39   58.77588   50.64170  13.8%     -    9s
     0     0   50.75381    0   39   58.77588   50.75381  13.6%     -    9s
     0     0   50.76152    0   40   58.77588   50.76152  13.6%     -    9s
     0     0   51.03578  

     0     0   57.54572    0   45   58.50852   57.54572  1.65%     -    9s
     0     0   57.55967    0   39   58.50852   57.55967  1.62%     -    9s
     0     0   57.69068    0   41   58.50852   57.69068  1.40%     -    9s
H    0     0                      58.3991767   57.69068  1.21%     -    9s
     0     0   57.69068    0   27   58.39918   57.69068  1.21%     -    9s
     0     0   58.28300    0    2   58.39918   58.28300  0.20%     -    9s

Cutting planes:
  Cover: 4
  Clique: 12
  MIR: 2
  StrongCG: 1
  GUB cover: 13

Explored 1 nodes (503 simplex iterations) in 9.47 seconds
Thread count was 4 (of 4 available processors)

Solution count 9: 58.3992 58.5085 66.1229 ... 160.492

Optimal solution found (tolerance 1.00e-04)
Best objective 5.839917665187e+01, best bound 5.839917665187e+01, gap 0.0000%
1.8 Simulation-General Ended!
1.8 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, 

...

4.0 Simulation-General Ended!
4.0 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x7ef39254
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 365.5653951
Presolve removed 15 rows and 199464 columns (presolve time = 10s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 9.98s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 299.2090077
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.0000

       0    0.0000000e+00   2.100000e+01   0.000000e+00     10s
      85    8.9603286e+01   0.000000e+00   0.000000e+00     10s

Root relaxation: objective 8.960329e+01, 85 iterations, 0.04 seconds
Total elapsed time = 10.26s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   89.60329    0   30  313.80457   89.60329  71.4%     -   10s
H    0     0                     214.5816472   89.60329  58.2%     -   10s
H    0     0                     170.8442010   89.60329  47.6%     -   10s
H    0     0                     168.4670386   89.60329  46.8%     -   10s
     0     0  104.33960    0   34  168.46704  104.33960  38.1%     -   10s
H    0     0                     158.0012185  104.33960  34.0%     -   10s
     0     0  107.53944    0   33  158.00122  107.53944  31.9%     -   10s
     0     0  124.85223    0   57  158.00122  124.85223  21.0%     -   10s
     0     0  124.8522


     0     0   93.77088    0   30  328.40013   93.77088  71.4%     -   10s
H    0     0                     224.5621889   93.77088  58.2%     -   10s
H    0     0                     178.7904429   93.77088  47.6%     -   10s
H    0     0                     176.3027148   93.77088  46.8%     -   10s
     0     0  109.19261    0   34  176.30271  109.19261  38.1%     -   10s
H    0     0                     165.3501124  109.19261  34.0%     -   10s
     0     0  112.54128    0   33  165.35011  112.54128  31.9%     -   10s
     0     0  130.65931    0   57  165.35011  130.65931  21.0%     -   11s
     0     0  130.65931    0   31  165.35011  130.65931  21.0%     -   11s
     0     0  130.65931    0   49  165.35011  130.65931  21.0%     -   11s
H    0     0                     165.3071761  130.65931  21.0%     -   11s
     0     0  132.69063    0   55  165.30718  132.69063  19.7%     -   11s
     0     0  133.12186    0   53  165.30718  133.12186  19.5%     -   11s
     0     0  133.12186 

     0     0  136.46639    0   57  172.69901  136.46639  21.0%     -   10s
     0     0  136.46639    0   31  172.69901  136.46639  21.0%     -   11s
     0     0  136.46639    0   49  172.69901  136.46639  21.0%     -   11s
H    0     0                     172.6541618  136.46639  21.0%     -   11s
     0     0  138.58799    0   55  172.65416  138.58799  19.7%     -   11s
     0     0  139.03839    0   53  172.65416  139.03839  19.5%     -   11s
     0     0  139.03839    0   54  172.65416  139.03839  19.5%     -   11s
     0     0  148.75999    0   39  172.65416  148.75999  13.8%     -   11s
     0     0  149.08931    0   39  172.65416  149.08931  13.6%     -   11s
     0     0  149.11195    0   40  172.65416  149.11195  13.6%     -   11s
     0     0  149.79366    0   49  172.65416  149.79366  13.2%     -   11s
H    0     0                     154.1374032  149.79366  2.82%     -   11s
H    0     0                     153.4448638  149.79366  2.38%     -   11s
     0     0  150.20667  

     0     0  144.96046    0   55  180.00115  144.96046  19.5%     -   11s
     0     0  155.09020    0   39  180.00115  155.09020  13.8%     -   11s
     0     0  155.42596    0   37  180.00115  155.42596  13.7%     -   11s
     0     0  155.44968    0   38  180.00115  155.44968  13.6%     -   11s
     0     0  156.02963    0   50  180.00115  156.02963  13.3%     -   11s
H    0     0                     159.4190029  156.02963  2.13%     -   11s
     0     0  156.51008    0   28  159.41900  156.51008  1.82%     -   11s
     0     0  156.58850    0   43  159.41900  156.58850  1.78%     -   11s
     0     0  156.58946    0   43  159.41900  156.58946  1.77%     -   11s
     0     0  156.80084    0   45  159.41900  156.80084  1.64%     -   11s
H    0     0                     158.9755364  156.80084  1.37%     -   12s
     0     0  156.80084    0   26  158.97554  156.80084  1.37%     -   12s
     0     0  158.15933    0   43  158.97554  158.15933  0.51%     -   12s
     0     0  158.65928  

In [4]:
import os
import numpy

N = 22

File = open("Trad-Solution.txt", mode='r')
temp = []
while True:
    Line = File.readline()
    if Line:
        Line = Line[:-2]
        Line = list(map(float,Line.split('\t')))
        Line = Line[1:]
        temp.append(Line)
    else:
        break
File.close()
print("inbound service time")
print(numpy.average(temp, axis=0)[0:N])
print(numpy.max(temp, axis=0)[0:N])
print(numpy.min(temp, axis=0)[0:N])
print("outbound service time")
print(numpy.average(temp, axis=0)[N:(2*N)])
print(numpy.max(temp, axis=0)[N:(2*N)])
print(numpy.min(temp, axis=0)[N:(2*N)])
print("base stock")
print(numpy.average(temp, axis=0)[(2*N):(3*N)])
print(numpy.max(temp, axis=0)[(2*N):(3*N)])
print(numpy.min(temp, axis=0)[(2*N):(3*N)])

inbound service time
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
outbound service time
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
base stock
[0.12195122 0.         1.41463415 1.56097561 1.56097561 1.
 1.         1.         2.70731707 1.56097561 0.         1.09756098
 1.         0.         0.         0.         1.82926829 3.26829268
 1.56097561 1.41463415 0.75609756 0.        ]
[1. 0. 2. 2. 2. 1. 1. 1. 3. 2. 0. 2. 1. 0. 0. 0. 2. 4. 2. 2. 1. 0.]
[0. 0. 1. 1. 1. 1. 1. 1. 2. 1. 0. 1. 1. 0. 0. 0. 1. 3. 1. 1. 0. 0.]
